이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.5.1 py4j==0.10.9.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 10.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=f442d742d67948ded05f6658d3a3d92f2954fcf387f0da9699c3c5d64b560021
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [1]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/orders.csv

--2024-06-22 00:37:56--  https://s3-geospatial.s3.us-west-2.amazonaws.com/orders.csv
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.218.237.201, 3.5.76.178, 3.5.76.198, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.218.237.201|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89951 (88K) [text/csv]
Saving to: ‘orders.csv.1’

orders.csv.1        100%[===================>]  87.84K   272KB/s    in 0.3s    

2024-06-22 00:37:57 (272 KB/s) - ‘orders.csv.1’ saved [89951/89951]



**Spark Session** 

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark Hive") \
    .config("spark.jars", "/home/sangwon/.local/lib/python3.10/site-packages/pyspark/jars/redshift-jdbc42-2.1.0.14.jar") \
    .enableHiveSupport() \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/22 00:38:33 WARN Utils: Your hostname, LAPTOP-8CK94L0D resolves to a loopback address: 127.0.1.1; using 172.21.9.191 instead (on interface eth0)
24/06/22 00:38:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/06/22 00:38:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
!ls -tl

total 368
-rw-r--r-- 1 sangwon sangwon 18345 Jun 22 00:38 PySpark_Hive_테이블.ipynb
-rw-r--r-- 1 sangwon sangwon 23991 Jun 22 00:36 PySpark_SQL_사용자별로_처음_채널과_마지막_채널_알아내기.ipynb
-rw-r--r-- 1 sangwon sangwon 24437 Jun 22 00:31 PySpark_SQL_월별_채널별_매출과_방문자_정보.ipynb
-rw-r--r-- 1 sangwon sangwon 16185 Jun 22 00:28 PySpark_SQL_총_매출이_가장_많은_사용자_10명_찾기.ipynb
-rw-r--r-- 1 sangwon sangwon 29758 Jun 21 23:49 PySpark_SQL_JOIN.ipynb
-rw-r--r-- 1 sangwon sangwon 49067 Jun 21 23:29 PySpark_UDF.ipynb
drwxr-xr-x 2 sangwon sangwon  4096 Jun 21 23:29 spark-warehouse
-rw-r--r-- 1 sangwon sangwon 16979 Jun 20 01:17 PySpark_유닛_테스트.ipynb
drwxr-xr-x 2 sangwon sangwon  4096 Jun 20 01:17 unittest
-rw-r--r-- 1 sangwon sangwon 89951 Apr 25  2022 orders.csv
-rw-r--r-- 1 sangwon sangwon 89951 Apr 25  2022 orders.csv.1


In [3]:
# Read in data and create a DataFrame
df = spark.read.csv("orders.csv", inferSchema=True, header=True, sep ='\t')

In [4]:
df.show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [5]:
spark.sql("CREATE DATABASE IF NOT EXISTS TEST_DB")
spark.sql("USE TEST_DB")

24/06/22 00:39:05 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/06/22 00:39:05 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/06/22 00:39:17 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/06/22 00:39:17 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore sangwon@127.0.1.1
24/06/22 00:39:17 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
24/06/22 00:39:17 WARN ObjectStore: Failed to get database test_db, returning NoSuchObjectException
24/06/22 00:39:17 WARN ObjectStore: Failed to get database test_db, returning NoSuchObjectException
24/06/22 00:39:17 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
24/06/22 00:39:17 WARN ObjectStore: Failed to get database test_db, returning NoSuch

DataFrame[]

In [6]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|  test_db|
+---------+



In [7]:
!ls -tl

total 288
drwxr-xr-x 3 sangwon sangwon  4096 Jun 22 00:39 spark-warehouse
drwxr-xr-x 5 sangwon sangwon  4096 Jun 22 00:39 metastore_db
-rw-r--r-- 1 sangwon sangwon   814 Jun 22 00:39 derby.log
-rw-r--r-- 1 sangwon sangwon 18345 Jun 22 00:38 PySpark_Hive_테이블.ipynb
-rw-r--r-- 1 sangwon sangwon 23991 Jun 22 00:36 PySpark_SQL_사용자별로_처음_채널과_마지막_채널_알아내기.ipynb
-rw-r--r-- 1 sangwon sangwon 24437 Jun 22 00:31 PySpark_SQL_월별_채널별_매출과_방문자_정보.ipynb
-rw-r--r-- 1 sangwon sangwon 16185 Jun 22 00:28 PySpark_SQL_총_매출이_가장_많은_사용자_10명_찾기.ipynb
-rw-r--r-- 1 sangwon sangwon 29758 Jun 21 23:49 PySpark_SQL_JOIN.ipynb
-rw-r--r-- 1 sangwon sangwon 49067 Jun 21 23:29 PySpark_UDF.ipynb
-rw-r--r-- 1 sangwon sangwon 16979 Jun 20 01:17 PySpark_유닛_테스트.ipynb
drwxr-xr-x 2 sangwon sangwon  4096 Jun 20 01:17 unittest
-rw-r--r-- 1 sangwon sangwon 89951 Apr 25  2022 orders.csv


In [8]:
df.write.saveAsTable("TEST_DB.orders", mode="overwrite")

24/06/22 00:39:38 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/06/22 00:39:38 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/06/22 00:39:38 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/06/22 00:39:38 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [9]:
!ls -tl spark-warehouse/test_db.db/orders/

total 24
-rw-r--r-- 1 sangwon sangwon     0 Jun 22 00:39 _SUCCESS
-rw-r--r-- 1 sangwon sangwon 23051 Jun 22 00:39 part-00000-3b20e2b1-ea93-4360-a241-c86ed89a203e-c000.snappy.parquet


In [10]:
spark.sql("SELECT * FROM TEST_DB.orders").show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [11]:
spark.table("TEST_DB.orders").show(5)

+------------+--------------------+
|    order_id|               items|
+------------+--------------------+
|860196503764|[{"name": "DAILY ...|
|860292645076|[{"name": "DAILY ...|
|860320956628|[{"name": "DAILY ...|
|860321513684|[{"name": "DAILY ...|
|862930665684|[{"name": "DAILY ...|
+------------+--------------------+
only showing top 5 rows



In [12]:
!ls -tl metastore_db/

total 28
drwxr-xr-x 2 sangwon sangwon 4096 Jun 22 00:39 seg0
-rw-r--r-- 1 sangwon sangwon  952 Jun 22 00:39 service.properties
-rw-r--r-- 1 sangwon sangwon  608 Jun 22 00:39 README_DO_NOT_TOUCH_FILES.txt
drwxr-xr-x 2 sangwon sangwon 4096 Jun 22 00:39 log
-rw-r--r-- 1 sangwon sangwon   38 Jun 22 00:39 db.lck
-rw-r--r-- 1 sangwon sangwon    4 Jun 22 00:39 dbex.lck
drwxr-xr-x 2 sangwon sangwon 4096 Jun 22 00:39 tmp


In [13]:
spark.catalog.listTables()

[Table(name='orders', catalog='spark_catalog', namespace=['test_db'], description=None, tableType='MANAGED', isTemporary=False)]

In [14]:
spark.sql("""
    DROP TABLE IF EXISTS TEST_DB.orders_count;
""")

DataFrame[]

In [15]:
spark.sql("""
    CREATE TABLE TEST_DB.orders_count AS 
    SELECT order_id, COUNT(1) as count 
    FROM TEST_DB.orders
    GROUP BY 1""")

24/06/22 00:40:32 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/06/22 00:40:33 WARN HiveMetaStore: Location: file:/home/sangwon/devcourse/spark/examples/pyspark_example/chapter3/spark-warehouse/test_db.db/orders_count specified for non-external table:orders_count


DataFrame[]

In [16]:
spark.catalog.listTables()

[Table(name='orders', catalog='spark_catalog', namespace=['test_db'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='orders_count', catalog='spark_catalog', namespace=['test_db'], description=None, tableType='MANAGED', isTemporary=False)]

In [17]:
!ls -tl spark-warehouse/test_db.db/

total 8
drwxr-xr-x 2 sangwon sangwon 4096 Jun 22 00:40 orders_count
drwxr-xr-x 2 sangwon sangwon 4096 Jun 22 00:39 orders


In [18]:
spark.sql("SELECT * FROM test_db.orders_count LIMIT 10").show()

+-------------+-----+
|     order_id|count|
+-------------+-----+
|1209310019796|    1|
|1253925257428|    1|
|1314797846740|    1|
|1370637402324|    1|
|1967690285268|    1|
|1971226443988|    1|
|2106987970772|    1|
|2135281533140|    1|
|1271771070676|    1|
|1713331765460|    1|
+-------------+-----+

